# Testing notebook

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
np.random.seed(1234)

import sys
sys.path.append("../src")
from utils import data as udata
from utils import dists as udists
from utils import misc as u
from truth import mask_truths
from predictors import make_predictor
import losses
from pymmwr import Epiweek
from tqdm import tqdm, trange
import ledge.merge as merge
import ledge.update as update
from functools import partial
import json
import os.path as path
import inspect

In [5]:
EXP_DIR = "../data/processed/cdc-flusight-ensemble/"
OUTPUT_DIR = "../models/cdc-flusight-ensemble/"
TARGET = "4-ahead"
MAX_LAG = 29
# REGIONS = ["nat", *[f"hhs{i}" for i in range(1, 11)]]
REGIONS = ["nat"] # Speed up!
TESTING_SEASONS = list(range(2014, 2017))
LOSS_FN = losses.ploss

In [4]:
components = [udata.Component(EXP_DIR, m) for m in udata.available_models(EXP_DIR)]
ad = udata.ActualData(EXP_DIR)

# Evaluation

In [6]:
def evaluate(predictor, loss_fn=losses.ploss):
    """
    Evaluate the predictor over all testing seasons and regions and return mean score
    """
    
    first_losses = []
    final_losses = []
    
    with tqdm(total=len(TESTING_SEASONS) * len(REGIONS)) as pbar:
        for season in TESTING_SEASONS:
            for region in REGIONS:
                truths = [ad.get(TARGET, region, season, lag=l) for l in range(MAX_LAG + 1)]
                c_preds = [cmp.get(TARGET, region, season) for cmp in components]
            
                first_truth = merge.earliest(truths)
                final_truth = merge.latest(truths)
                pred, _ = predictor(truths, c_preds)
                first_losses.append(float(loss_fn(pred, first_truth).mean()))
                final_losses.append(float(loss_fn(pred, final_truth).mean()))
                pbar.update()

    return {
        "first_loss": np.mean(first_losses),
        "final_loss": np.mean(final_losses)
    }

# Tuning

In [7]:
# Merge strategy
MERGE_FN = merge.latest

## Individuals

In [8]:
for idx, cmp in enumerate(components):
    update_fn = partial(update.pick, index=idx)
    losses = evaluate(make_predictor(LOSS_FN, MERGE_FN, update_fn), LOSS_FN)
    print(f"{cmp.name}: {losses['first_loss'], losses['final_loss']}")

  0%|          | 0/3 [00:00<?, ?it/s]

CU-BMA: (0.9566286783335394, 0.9559738634665339)


  0%|          | 0/3 [00:00<?, ?it/s]

CU-EAKFC_SEIRS: (0.918566493098916, 0.9243592063279785)


  0%|          | 0/3 [00:00<?, ?it/s]

...

ReichLab-SARIMA1: (0.9436938567352796, 0.9430457113784835)


  0%|          | 0/3 [00:00<?, ?it/s]

ReichLab-SARIMA2: (0.9370948668362896, 0.9367517719845443)


100%|██████████| 3/3 [00:11<00:00,  3.98s/it]

UTAustin-edm: (0.9506847658261887, 0.9517426810754532)


## Mean

In [9]:
losses = evaluate(make_predictor(LOSS_FN, MERGE_FN, update.noop), LOSS_FN)
print(f"{losses['first_loss'], losses['final_loss']}")

100%|██████████| 3/3 [00:12<00:00,  4.03s/it]

(0.9361819616639192, 0.9374684827895591)


## Dem

In [10]:
# Load the weights from DEM training done in tracking-ensemble repository
with open(path.join(OUTPUT_DIR, TARGET, "dem-weight-ensemble.json")) as fp:
    w_vec = json.load(fp)["fit_params"]["weights"]
    models = [cmp.name for cmp in components]
    weights = xr.DataArray(w_vec, dims="model", coords={ "model": models })
    
update_fn = partial(update.noop, init_weights=weights)
losses = evaluate(make_predictor(LOSS_FN, MERGE_FN, update_fn), LOSS_FN)
print(f"{losses['first_loss'], losses['final_loss']}")

100%|██████████| 3/3 [00:12<00:00,  4.14s/it]

(0.918741770779441, 0.9186077575437407)


## Follow the leader

In [11]:
losses = evaluate(make_predictor(LOSS_FN, MERGE_FN, update.ftl), LOSS_FN)
print(f"{losses['first_loss'], losses['final_loss']}")

100%|██████████| 3/3 [00:12<00:00,  4.23s/it]

(0.9130408818963921, 0.9093131505186269)


## MW

In [13]:
update_fn = partial(update.mw, eta=0.9)
losses = evaluate(make_predictor(LOSS_FN, MERGE_FN, update_fn), LOSS_FN)
print(f"{losses['first_loss'], losses['final_loss']}")

100%|██████████| 3/3 [00:13<00:00,  4.36s/it]

(0.92823693466531, 0.9183584201127551)


## Hedge

In [14]:
update_fn = partial(update.hedge, eta=24.1)
losses = evaluate(make_predictor(LOSS_FN, MERGE_FN, update_fn), LOSS_FN)
print(f"{losses['first_loss'], losses['final_loss']}")

100%|██████████| 3/3 [00:13<00:00,  4.63s/it]

(0.9161306532977197, 0.9111996125226796)


## Fixed share

In [15]:
update_fn = partial(update.fixed_share, eta=24.1, alpha=0.1)
losses = evaluate(make_predictor(LOSS_FN, MERGE_FN, update_fn), LOSS_FN)
print(f"{losses['first_loss'], losses['final_loss']}")

100%|██████████| 3/3 [00:29<00:00,  9.80s/it]

(0.9110845425737596, 0.9116343423639858)
